In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

**Proceso:**
- Una calle y un sentido: Cada partícula avanza con probabilidad p solo si esta despejado.

**Queremos:**
- Función que simule TASEP: recibe probabilidad de avanzar, si es secuencial o paralelo, tamaño de la calle, cantidad inicial de particulas y tasa de generación de particulas.

In [2]:
class Particula:

    """
    Clase que representa una partícula en un sistema.
    Atributos:
    - posicion (int): La posición actual de la partícula.
    - bloqueado (bool): Indica si la partícula está bloqueada o no.
    Métodos:
    - __init__(posicion: int, bloqueado: bool): Inicializa una nueva instancia de la clase Particula.
    - avanzar(): Avanza la partícula una posición hacia adelante.
    - __str__(): Devuelve una representación en cadena de la partícula como str.
    """

    def __init__(self, posicion: int, bloqueado: bool) -> None:
        self.posicion = posicion
        self.bloqueado = bloqueado
    
    def avanzar(self) -> None:
        self.posicion += 1

    def __str__(self) -> str:
        return f'Posición: {self.posicion}, Bloqueado: {self.bloqueado}'


In [72]:
class Interseccion():

    def __init__(self, posicion: tuple, bloqueado: list, calles: tuple) -> None:
        self.posicion = posicion
        self.bloqueado = bloqueado
        self.calles = calles

    def __str__(self) -> str:
        return f'Posición: {self.posicion}, Bloqueado: {self.bloqueado}'

    def update_bloqueo(self, direccion):
        self.calles[direccion].update_bloqueo()

In [80]:
class Calle(list):

    """
    Clase que representa una calle en un modelo de simulación TASEP.
    Métodos:
    - __init__(*args): Constructor de la clase.
    - agregar_particula_inicio(): Agrega una partícula al inicio de la calle.
    - update_bloqueo_casilla(i: int): Actualiza el estado de bloqueo de una casilla en la calle.
    - update_bloqueo(): Actualiza el estado de bloqueo de todas las casillas en la calle.
    - update_secuencial(p: float): Actualiza la posición de las partículas en la calle de forma secuencial.
    - update_paralelo(p: float): Actualiza la posición de las partículas en la calle de forma paralela.
    - plot(): Grafica la posición de las partículas en la calle.
    - animate(n: int, p: float, mode: str): Genera una animación de la simulación.
    - __str__(): Retorna una representación en cadena de la calle.
    """

    def __init__(self, *args, direccion: int, intersecciones: list, posicion: int) -> None:
        super().__init__(*args)
        self.direccion = direccion
        self.intersecciones = intersecciones
        self.posicion = posicion

    def agregar_particula_inicio(self) -> None:
        self.insert(0, Particula(0, self[0].posicion == 1))

    def agregar_interseccion(self, interseccion: Interseccion) -> None:
        self.intersecciones.append(interseccion)

    def update_estado_intersecciones(self) -> None:
        for interseccion in self.intersecciones:
            for particula in self:
                if particula.posicion == interseccion.posicion[(self.direccion + 1) % 2]:
                    interseccion.bloqueado[self.direccion] = True
                    for particula_otra_calle in interseccion.calles[(self.direccion + 1) % 2]:
                        if particula_otra_calle.posicion + 1 == interseccion.posicion[self.direccion]:
                            particula_otra_calle.bloqueado = True
                            break
                    break

    def update_bloqueo_casilla(self, i: int) -> None:
        self[i].bloqueado = self[i+1].posicion == self[i].posicion + 1
        if not self[i].bloqueado:
            for interseccion in self.intersecciones:
                self[i].bloqueado = self[i].posicion + 1 == interseccion.posicion[self.direccion] and interseccion.bloqueado[(self.direccion + 1) % 2]

    def update_bloqueo(self) -> None:
        for i in range(len(self) - 1):
            self.update_bloqueo_casilla(i)
    
    def update_secuencial(self, p: float) -> None:
        for i in range(len(self)-1, -1, -1):
            if not self[i].bloqueado and np.random.rand() < p:
                self[i].avanzar()
                if i != len(self) - 1:
                    self.update_bloqueo_casilla(i)
                if i != 0:
                    self[i-1].bloqueado = False
        self.update_estado_intersecciones()

    def update_paralelo(self, p: float) -> None:
        for i in range(len(self)-1, -1, -1):
            if not self[i].bloqueado:
                if np.random.rand() < p:
                    self[i].avanzar()
        self.update_bloqueo()
        self.update_estado_intersecciones()

    def plot(self) -> None:
        plt.plot([p.posicion for p in self], [0 for _ in self], 'ro')
        plt.show()

    def animate(self, n: int, p: float, mode: str) -> None:
        fig, ax = plt.subplots()
        line, = ax.plot([], [], 'ro')

        def init():
            ax.set_xlim(0, len(self))
            ax.set_ylim(0, 1)
            return line,

        def update(frame):
            if mode == 'secuencial':
                self.update_secuencial(p)
            elif mode == 'paralelo':
                self.update_paralelo(p)
            line.set_data([p.posicion for p in self], [0 for _ in self])
            ax.set_xlim(0, len(self))
            return line,

        ani = animation.FuncAnimation(fig, update, frames=n, init_func=init, blit=True)
        ani.save('animation.gif', writer='pillow')

    def __str__(self) -> str:
        return '\n'.join([str(p) for p in self])

In [81]:
class Calles():

    def __init__(self, calles: list) -> None:
        self.calles = calles
        self.calles_x = [c for c in calles if c.direccion == 0]
        self.calles_y = [c for c in calles if c.direccion == 1]
        self.intersecciones = []

    def update_intersecciones(self) -> None:
        for calle_x in self.calles_x:
            for calle_y in self.calles_y:
                choque_x = False
                choque_y = False
                for particula in calle_x:
                    if particula.posicion == calle_y.posicion:
                        choque_x = True
                        break
                for particula in calle_y:
                    if particula.posicion == calle_x.posicion:
                        choque_y = True
                        break
                if choque_x and choque_y:
                    raise ValueError('Hay un choque')         
                interseccion = Interseccion((calle_x.posicion, calle_y.posicion), [choque_x, choque_y], (calle_x, calle_y))
                calle_x.agregar_interseccion(interseccion)
                calle_y.agregar_interseccion(interseccion)
                self.intersecciones.append(interseccion)

    def update_bloqueos(self) -> None:
        for calle in self.calles:
            calle.update_bloqueo()

    def update_estado_intersecciones(self) -> None:
        for calle in self.calles:
            calle.update_estado_intersecciones()

    def update_secuencial(self, p: float) -> None:
        for calle in self.calles:
            calle.update_secuencial(p)
            calle.update_bloqueo()

    def update_paralelo(self, p: float) -> None:
        calles_copy = self.calles.copy()
        random.shuffle(calles_copy)
        for calle in calles_copy:
            calle.update_paralelo(p)
            calle.udpate_bloqueo()

    def plot(self) -> None:
        for calle in self.calles:
            calle.plot()

In [82]:
calles = Calles([Calle([Particula(0,0), Particula(2,0)], direccion=0, intersecciones=[], posicion=3),
                 Calle([Particula(0,0), Particula(1,0)], direccion=1, intersecciones=[], posicion=4)])
calles.update_bloqueos()
calles.update_intersecciones()

In [83]:
for interseccion in calles.intersecciones:
    print(str(interseccion))

Posición: (3, 4), Bloqueado: [False, False]


In [84]:
print(str(calles.calles[1]))

Posición: 0, Bloqueado: True
Posición: 1, Bloqueado: 0


In [86]:
calles.update_secuencial(0.9)
print('-------------------')
print(str(calles.calles[0]))
print(str(calles.calles[1]))
for interseccion in calles.intersecciones:
    print(str(interseccion))
print('-------------------')

-------------------
Posición: 2, Bloqueado: False
Posición: 4, Bloqueado: 0
Posición: 1, Bloqueado: True
Posición: 2, Bloqueado: True
Posición: (3, 4), Bloqueado: [True, False]
-------------------
